In [ ]:
pip install llama-index-llms-openai

In [42]:
import pandas as pd
from pathlib import Path

data_dir = Path("./content/opportunities")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: content/opportunities/opportunities_transformed_description_only_clean.csv
processing file: content/opportunities/opportunities_transformed_no_description.csv


In [ ]:
tableinfo_dir = "Gov_Opportunties_TableInfo"
!mkdir {tableinfo_dir}

In [43]:
import os
import openai
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI
import json

with open('openai.secret.json', 'r') as file:
    secret = json.load(file)
    os.environ["OPENAI_API_KEY"] = secret['secret']
    openai.api_key = os.environ["OPENAI_API_KEY"]

class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise.
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-3.5-turbo"),
    prompt_template_str=prompt_str,
)

In [ ]:
pip install ipywidgets

In [44]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )


table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            print(df_str)
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)

In [48]:
table_infos.pop()
table_infos.pop(0)

TableInfo(table_name='Construction_Projects_Table', table_summary='A table containing information about various construction projects, including project descriptions, locations, and contract details.')

In [49]:
for table_info in table_infos:
  print("table summary for " + table_info.table_name)
  print(table_info.table_summary)



table summary for Construction_Projects_Table
A table containing information about various construction projects, including project descriptions, locations, and contract details.
table summary for Construction_Projects_Summary_Table
Summary of construction projects including project details, agency information, set-aside codes, response due dates, award information, and contact details.


In [ ]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re


# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()


engine = create_engine("sqlite:///:gov_opportunity.db")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

In [ ]:
pip install arize-phoenix

In [ ]:
pip install llama-index-callbacks-arize-phoenix

In [ ]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

In [ ]:
pip install llama-index

In [ ]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [99]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database_read = SQLDatabase(engine, max_string_length=5000)
sql_retriever = SQLRetriever(sql_database_read)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        print(table_info)
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context
        print(table_info + "\n")
        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
INFO  [sqlalchemy.engine.Engine] ROLLBACK
INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
INFO  [sqlalchemy.engine.Engine] SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_xinfo("Construction_Projects_Summary_Table")
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
INFO  [sqlalchemy.engine.Engine] PRAGMA main.table_xinfo("Construction_Projects_Table")
INFO  [sqlalchemy.engine.Engine] [raw sql] ()
INFO  [sqlalchemy.engine.

In [63]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.llms import ChatResponse
from llama_index.core.prompts.prompt_type import PromptType


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
import os
print(os.getcwd())
with open('prompts/text2sql_prompts.txt', 'r') as file:
    text2sql_prompt = PromptTemplate(file.read(), prompt_type=PromptType.TEXT_TO_SQL).partial_format(dialect=engine.dialect.name)
print(text2sql_prompt.template)

/Users/haiyangliu/Workspace/gov_opportunity_new
Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. 
For any date, always use the format like 2024-06-24T14:00:00-08:00.
For PlaceOfPerformanceCountry use the 3-letter country code, for example: USA, KOR.
For keyword search, always check for contain instead of exact matching. 
You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Quer

In [111]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Write a new line for each record found.\n"
    "If the SQL response is empty, tell no record is found\n"
    "If the query is to summarize, provide a concise and comprehensive summary (no more than 3 sentences) of the SQL response, otherwise give the response itself.\n"
    "\n"
    "Query: {query_str}\n"
    "\n"
    "SQL: {sql_query}\n"
    "\n"
    "SQL Response: {context_str}\n"
    "\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")


In [112]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [113]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [ ]:
pip install pyvis

In [ ]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)


# Save the network as "text2sql_dag.html"
net.write_html("text2sql_dag.html")


from IPython.display import display, HTML

# Read the contents of the HTML file
with open("text2sql_dag.html", "r") as file:
    html_content = file.read()

# Display the HTML content
display(HTML(html_content))

In [ ]:
from sqlalchemy import create_engine, Table, MetaData, select, inspect

def execute_query(engine, table_name):
    # Create an engine and connect to the database
    connection = engine.connect()
    metadata = MetaData()

    # Reflect the table from the database
    table = Table(table_name, metadata, autoload_with=engine)

    # Create a query
    query = select(table.c.ProjectID, table.c.Title).where(
        table.c.Title.like('%airport%'),
        table.c.PlaceOfPerformanceCountry == 'USA'
    ).order_by(table.c.ProjectID)

    # Execute the query
    result = connection.execute(query)
    records = result.fetchall()

    # Close the connection
    connection.close()

    return records

# Database and table details
table_name = "Construction_Projects_Summary_Table"  # Replace with your actual table name
record_limit = 10

# Fetch and print records
records = execute_query(engine, table_name)
for record in records:
    print(record)

In [66]:
response = qp.run(
    query="Can you list all the projects with airport in the title that is in the USA with a due date in June 2024"
)
print(str(response))

> Running module input with input: 
query: Can you list all the projects with airport in the title that is in the USA with a due date in June 2024

> Running module table_retriever with input: 
input: Can you list all the projects with airport in the title that is in the USA with a due date in June 2024

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='Construction_Projects_Summary_Table', context_str='Summary of construction projects including project details, agency information, set-aside codes, response due dates, awar...

Table 'Construction_Projects_Summary_Table' has columns: ProjectID (VARCHAR), Title (VARCHAR), NoticeID (VARCHAR), Department_Ind_Agency (VARCHAR), CGAC (INTEGER), Sub_Tier (VARCHAR), FPDS_Code (VARCHAR), Office (VARCHAR), AAC_Code (VARCHAR), PostedDate (VARCHAR), Type (VARCHAR), BaseType (VARCHAR), ArchiveType (VARCHAR), SetASideCode (VARCHAR), SetASide (VARCHAR), ResponseDueDate (VARCHAR), NaicsCode (INTEGER), Prod

In [114]:
response = qp.run(
    query="give me a summary of description of the project 507d079a58d74ddd9e5ee1e0f43032de"
)
print(str(response))

> Running module input with input: 
query: give me a summary of description of the project 507d079a58d74ddd9e5ee1e0f43032de

> Running module table_retriever with input: 
input: give me a summary of description of the project 507d079a58d74ddd9e5ee1e0f43032de

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='Construction_Projects_Summary_Table', context_str='Summary of construction projects including project details, agency information, set-aside codes, response due dates, awar...

Table 'Construction_Projects_Summary_Table' has columns: ProjectID (VARCHAR), Title (VARCHAR), NoticeID (VARCHAR), Department_Ind_Agency (VARCHAR), CGAC (INTEGER), Sub_Tier (VARCHAR), FPDS_Code (VARCHAR), Office (VARCHAR), AAC_Code (VARCHAR), PostedDate (VARCHAR), Type (VARCHAR), BaseType (VARCHAR), ArchiveType (VARCHAR), SetASideCode (VARCHAR), SetASide (VARCHAR), ResponseDueDate (VARCHAR), NaicsCode (INTEGER), ProductServiceCode (VARCHAR), PlaceOfPerformanceSt

INFO  [sqlalchemy.engine.Engine] BEGIN (implicit)
INFO  [sqlalchemy.engine.Engine] SELECT Description
FROM Construction_Projects_Table
WHERE ProjectID = '507d079a58d74ddd9e5ee1e0f43032de';
INFO  [sqlalchemy.engine.Engine] [cached since 4851s ago] ()
INFO  [sqlalchemy.engine.Engine] COMMIT


> Running module sql_output_parser with input: 
response: assistant: SELECT Description
FROM Construction_Projects_Table
WHERE ProjectID = '507d079a58d74ddd9e5ee1e0f43032de'; 
SQLResult: Description: Renovation of existing office building including HVAC upgr...

this is a new information!!!
SELECT Description
FROM Construction_Projects_Table
WHERE ProjectID = '507d079a58d74ddd9e5ee1e0f43032de'; 
SQLResult: Description: Renovation of existing office building including HVAC upgrades and interior remodeling.
Answer: Renovation of existing office building including HVAC upgrades and interior remodeling.
> Running module sql_retriever with input: 
input: SELECT Description
FROM Construction_Projects_Table
WHERE ProjectID = '507d079a58d74ddd9e5ee1e0f43032de';

> Running module response_synthesis_prompt with input: 
query_str: give me a summary of description of the project 507d079a58d74ddd9e5ee1e0f43032de
sql_query: SELECT Description
FROM Construction_Projects_Table
WHERE ProjectID = '507d0

Fields that can cause problem because of inconsistencies, need to be included in the prompt
1. Date format
2. Department: should be in the format of DEPT OF DEFENSE
3. Use "like" instead of "equal"
4. when response is empty, don't make up answers